In [ ]:
# Dashboard

In [ ]:
# realizar limpieza de la base de datos

In [2]:
import pandas as pd

In [3]:
# cargar el archivo

In [4]:
archivo = "Casos_2025.xlsx"
df = pd.read_excel(archivo)

C:\Users\T\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
# primeras filas

In [9]:
print(df.head())

   ID Caso ID Caso Relacionado   Año  Mes  Día Código DANE de Municipio  \
0   152888                 NaN  2004    4    0                    05585   
1   207556                 NaN  2001    4   25                    05756   
2   209268                 NaN     0    0    0                    05234   
3   225057                 NaN  1988    0    0                    68689   
4   228661                 NaN  1998    2   15                    66456   

                Municipio Departamento                Región    Modalidad  \
0             PUERTO NARE    ANTIOQUIA       MAGDALENA MEDIO   PERSUASIÓN   
1                  SONSON    ANTIOQUIA    ORIENTE ANTIOQUEÑO   PERSUASIÓN   
2                 DABEIBA    ANTIOQUIA  OCCIDENTE ANTIOQUEÑO  DESCONOCIDA   
3  SAN VICENTE DE CHUCURI    SANTANDER       MAGDALENA MEDIO     COACCIÓN   
4                MISTRATO    RISARALDA          EJE CAFETERO   PERSUASIÓN   

   ... Lesionados Civiles Pillaje  Tortura  Violencia Basada en Género  \
0  ...      

In [10]:
# columnas disponibles

In [11]:
print(df.columns)

Index(['ID Caso', 'ID Caso Relacionado', 'Año', 'Mes', 'Día',
       'Código DANE de Municipio', 'Municipio', 'Departamento', 'Región',
       'Modalidad', 'Presunto Responsable', 'Descripción Presunto Responsable',
       'Abandono o Despojo Forzado de Tierras', 'Amenaza o Intimidación',
       'Ataque Contra Misión Médica',
       'Confinamiento o Restricción a la Movilidad', 'Desplazamiento Forzado',
       'Extorsión', 'Lesionados Civiles', 'Pillaje', 'Tortura',
       'Violencia Basada en Género', 'Otro Hecho Simultáneo',
       'Total de Víctimas del Caso', 'Forma de Vinculación',
       'Tipo de Vinculación', 'Latitud', 'Longitud'],
      dtype='object')


In [12]:
# columnas ha usar 

In [13]:
columnas_utiles = [
    'ID Caso','Año','Departamento','Municipio','Modalidad','Presunto Responsable','Desplazamiento Forzado','Confinamiento o Restricción a la Movilidad','Extorsión','Tortura','Total de Víctimas del Caso'
]
df=df[columnas_utiles]

In [14]:
# renombrar columnas para facilitar el analisis

In [15]:
df = df.rename(columns={
    'Presunto Responsable': 'Responsable',
    'Desplazamiento Forzado': 'Desplazamiento',
    'Confinamiento o Restricción a la Movilidad': 'Confinamiento',
    'Amenaza o Intimidacion':'Amenaza',
    'Total de Víctimas del Caso': 'Victimas'
})

In [16]:
print(df.isnull().sum())

ID Caso           0
Año               0
Departamento      0
Municipio         0
Modalidad         0
Responsable       0
Desplazamiento    0
Confinamiento     0
Extorsión         0
Tortura           0
Victimas          0
dtype: int64


In [17]:
# eliminar filas vacias

In [18]:
df = df.dropna(how='all')

In [19]:
# ver duplicados

In [20]:
duplicados = df.duplicated()
print(f"Filas duplicadas: {duplicados.sum()}")

Filas duplicadas: 0


In [21]:
# ver el tipo de dato

In [22]:
print(df.dtypes)

ID Caso            int64
Año                int64
Departamento      object
Municipio         object
Modalidad         object
Responsable       object
Desplazamiento     int64
Confinamiento      int64
Extorsión          int64
Tortura            int64
Victimas           int64
dtype: object


In [ ]:
# verificar tipos de datos

In [24]:
print(df['Desplazamiento'].unique())
print(df['Confinamiento'].unique())
print(df['Extorsión'].unique())
print(df['Tortura'].unique())


[0 1]
[0 1]
[0 1]
[ 0  1  3  2 10 11]


In [ ]:
# cambiar tipos de datos 

In [26]:
df['Desplazamiento'] = df['Desplazamiento'].astype(int)
df['Confinamiento'] = df['Confinamiento'].astype(int)
df['Extorsión'] = df['Extorsión'].astype(int)
df['Tortura'] = df['Tortura'].astype(int)

In [ ]:
# crear columnas derivadas

In [27]:
def Tipo_violencia(row):
    tipos = []
    if row['Desplazamiento']==1:
        tipos.append('Desplazamiento')
    if row['Confinamiento']==1:
        tipos.append('Confinamiento')
    if row['Extorsión']==1:
        tipos.append('Extorsión')
    if row['Tortura']>0:
        tipos.append('Tortura')
    return','.join(tipos)

df['Tipo_violencia']=df.apply(Tipo_violencia, axis=1)

In [32]:
df['Num_tipo_violencia'] = (
    df['Desplazamiento'] +
    df['Confinamiento'] +
    df['Extorsión'] +
    (df['Tortura'] > 0).astype(int)
)

In [34]:
def Clasificar_gravedad(row):
   if row['Victimas'] >= 10 or row['Tortura'] > 0 :
       return 'Alta'
   elif row['Victimas'] >= 5 :
       return 'Media'
   else:
       return 'Baja'

df['Gravedad'] = df.apply(Clasificar_gravedad, axis=1)

In [35]:
df['Decada'] = (df['Año']//10)*10

In [ ]:
# guardar los datos

In [36]:
df.to_csv("datos_transformados.csv", index=False)

In [ ]:
# estandarizar los valores (Departamento,modalidad,Responsable)

In [38]:
print(df['Departamento'].unique())

['ANTIOQUIA' 'SANTANDER' 'RISARALDA' 'META' 'TOLIMA' 'CALDAS' 'CORDOBA'
 'VAUPES' 'NARIÑO' 'HUILA' 'CHOCO' 'SIN INFORMACION' 'EXTERIOR' 'VICHADA'
 'GUAVIARE' 'GUAINIA' 'AMAZONAS'
 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA' 'PUTUMAYO'
 'CASANARE' 'ARAUCA' 'VALLE DEL CAUCA' 'SUCRE' 'QUINDIO'
 'NORTE DE SANTANDER' 'MAGDALENA' 'CESAR' 'LA GUAJIRA' 'CAQUETA'
 'CUNDINAMARCA' 'CAUCA' 'BOYACA' 'BOLIVAR' 'BOGOTA, D. C.' 'ATLANTICO']


In [41]:
df['Departemento'] = df['Departamento'].replace({
    'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA': 'SAN ANDRES,PROVIDENCIA Y SANTA CATALINA',
    'BOGOTA, D.C.':'BOGOTA D.C.'
})

In [42]:
print(df['Modalidad'].unique())

['PERSUASIÓN' 'DESCONOCIDA' 'COACCIÓN' 'ENGAÑO' 'OTRA ¿CUÁL?']


In [43]:
df['Modalidad'] = df['Modalidad'].replace({
    'DESCONOCIDAD': 'SIN INFORMACION',
    'OTRA ¿CUÁL?': 'OTRA'
})

In [45]:
print(df['Responsable'].unique())

['GRUPO PARAMILITAR' 'GUERRILLA' 'DESCONOCIDO'
 'GRUPO ARMADO NO IDENTIFICADO' 'AGENTE DEL ESTADO'
 'GRUPO POSDESMOVILIZACIÓN' 'BANDOLERISMO' 'OTRO ¿CUÁL?'
 'CRIMEN ORGANIZADO' 'AGENTE DEL ESTADO - GRUPO PARAMILITAR']


In [46]:
df['Respnsable'] = df['Responsable'].replace({
    'GRUPO ARMADO NO IDENTIFICADO': 'DESCONOCIDO',
    'BANDOLERISMO': 'OTRO',
    'OTRO ¿CUAL?': 'OTRO',
    'AGENTE DEL ESTADO - GRUPO PARAMILITAR': 'AGENTE DEL ESTADO'
})

In [ ]:
# indicadores clave

In [48]:
# 1. Total de casos 
total_casos = df['ID Caso'].nunique()

# 2. Total de víctimas
total_victimas = df['Victimas'].sum()

# 3. Promedio de víctimas por caso
promedio_victimas = df['Victimas'].mean()

# 4. Casos con violencia grave
casos_graves = df[df['Gravedad'] == 'Alta'].shape[0]

# 5. Casos con múltiples tipos de violencia
casos_multiviolencia = df[df['Num_tipo_violencia'] > 1].shape[0]

# 6. Departamentos afectados
departamentos_afectados = df['Departamento'].nunique()

# 7. Responsable más frecuente
responsable_top = df['Responsable'].value_counts().idxmax()

# 8. Número de responsables más comunes (top 5)
responsables_top_5 = df['Responsable'].value_counts().head(5)

# Mostrar resultados
print("🧾 Total de Casos:           ", total_casos)
print("👥 Total de Víctimas:        ", total_victimas)
print("📊 Promedio por Caso:        ", round(promedio_victimas, 2))
print("🔥 Casos con Gravedad Alta:  ", casos_graves)
print("⚠️ Casos con Violencia Múltiple:", casos_multiviolencia)
print("🗺️ Departamentos Afectados:  ", departamentos_afectados)
print("🎯 Responsable Más Frecuente:", responsable_top)
print("\n📌 Top 5 Responsables:\n", responsables_top_5)


🧾 Total de Casos:            17280
👥 Total de Víctimas:         18275
📊 Promedio por Caso:         1.06
🔥 Casos con Gravedad Alta:   39
⚠️ Casos con Violencia Múltiple: 14
🗺️ Departamentos Afectados:   35
🎯 Responsable Más Frecuente: DESCONOCIDO

📌 Top 5 Responsables:
 Responsable
DESCONOCIDO                     9314
GUERRILLA                       4329
GRUPO PARAMILITAR               1856
GRUPO ARMADO NO IDENTIFICADO    1498
GRUPO POSDESMOVILIZACIÓN         262
Name: count, dtype: int64


In [50]:
df.to_csv('resultado_final.csv', index=False)